In [1]:
import spacy
import pdfminer
import re
import os
import pandas as pd
import pdf2txt

/home/incipe/.local/lib/python3.6/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
/home/incipe/.local/lib/python3.6/site-packages/pdfminer/pdfdocument.py:22: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend


In [2]:
def convert_pdf(f):
    output_filename = os.path.basename(os.path.splitext(f)[0]) + ".txt"
    output_filepath = os.path.join("output/txt/", output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath]) # pdf to txt and save it in the given location
    print(output_filepath + " saved successfully!!!")
    return open(output_filepath).read()

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
result_dict = {"name":[], "phone": [], "email": [], "skills":[]}
names = []
phones = []
emails = []
skills = []

In [8]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

In [9]:
for file in os.listdir("resumes/"):
    if file.endswith(".pdf"):
        print("Reading....." + file)
        txt = convert_pdf(os.path.join("resumes/", file))
        parse_content(txt)

Reading.....Alisson ParkerCV.pdf
output/txt/Alisson ParkerCV.txt saved successfully!!!
Alisson Parker-Wright                                                                                                 10/10/1974
alli1414parks@mail.com
Extraction completed successfully!!!
Reading.....AshleyMilesCV.pdf
output/txt/AshleyMilesCV.txt saved successfully!!!
Ashley Miles
ashleymiles@memail.com
Extraction completed successfully!!!
Reading.....John DominicCV.pdf
output/txt/John DominicCV.txt saved successfully!!!
John Dominic
johndominic@mail.com
Extraction completed successfully!!!


In [11]:
result_dict["name"] = names
result_dict["phone"] = phones
result_dict["email"] = emails
result_dict["skills"] = skills
print(result_dict)

{'name': ['Alisson Parker-Wright                                                                                                 10/10/1974', 'Ashley Miles', 'John Dominic'], 'phone': ["['8569878511']", "['6592251422']", "['7877756411']"], 'email': [alli1414parks@mail.com, ashleymiles@memail.com, johndominic@mail.com], 'skills': ["{'tableau', 'python', 'java'}", "{'tableau', 'sql'}", "{'python', 'hadoop', 'java'}"]}


In [12]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Alisson Parker-Wright ...,['8569878511'],alli1414parks@mail.com,"{'tableau', 'python', 'java'}"
1,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'tableau', 'sql'}"
2,John Dominic,['7877756411'],johndominic@mail.com,"{'python', 'hadoop', 'java'}"


In [13]:
result_df.to_csv('output/csv/parsed_resumes.csv')

In [14]:
result_df.to_excel('output/csv/parsed_resumes.xlsx')